# Variational Autoencoders

In [ ]:
import subprocess
subprocess.run(["npm", "i", "--no-progress"], cwd="../widgets", check=True)
subprocess.run(["npm", "i", "--no-progress"], cwd="widget-wrappers", check=True)
subprocess.run(["bash", "build_wrapped_widgets.sh"], cwd="widget-wrappers", check=True)

In [ ]:
from vaewidgets import *  #
from dataset import generate_dataset
from util import map_tuple
from constants import size_range, hue_range

## Dataset explanation

In [ ]:
dataset_explanation()

## Train/validation set split

In [ ]:
valset_selection = AreaSelectionWidget()
valset_selection

In [ ]:
trainset_coords, valset_coords, trainset, valset = generate_dataset(
    size_range=size_range,
    hue_range=hue_range,
    valset_size_range=map_tuple(
        size_range, (valset_selection.x, valset_selection.x + valset_selection.width)
    ),
    valset_hue_range=map_tuple(
        hue_range, (valset_selection.y, valset_selection.y + valset_selection.height)
    ),
    num_samples=1000,
)

In [ ]:
dataset_visualization(trainset_coords, valset_coords, trainset, valset)